# Disaster Resource Query System
This notebook implements a complete pipeline to:
1. Fetch disaster resource data from an API.
2. Parse and process the data.
3. Generate and store embeddings for districts using FAISS.
4. Implement a query system with retrieval-augmented generation (RAG) using the Groq API.

**Key Features:**
- Integration with FAISS for similarity search.
- Use of Groq LLM API for context-aware answers.


## Step 1: Fetch and Store Data from API

In [ ]:
import requests
import json

def fetch_and_store_data(api_url, output_file):
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()

        with open(output_file, 'w') as file:
            json.dump(data, file, indent=4)

        print(f"Data successfully fetched from {api_url} and stored in {output_file}")
    except requests.exceptions.RequestException as e:
        print(f"Error while fetching data: {e}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")

# Example usage
api_url = "https://api.mlsakiit.com/resources"
output_file = "resources_data.json"
fetch_and_store_data(api_url, output_file)

## Step 2: Parse the Data

In [ ]:
import json

with open('resources_data.json', 'r') as f:
    data = json.load(f)

parsed_data = []

for feature in data.get('features', []):
    properties = feature.get('properties', {})
    geometry = feature.get('geometry', {})

    parsed_data.append({
        'district_name': properties.get('dist_name', 'Unknown'),
        'district_code': properties.get('dist_code', 'Unknown'),
        'water': properties.get('water', 0),
        'food_rations': properties.get('food_rations', 0),
        'medkits': properties.get('medkits', 0),
        'ammo': properties.get('ammo', 0),
        'camp_exists': properties.get('camp_exists', False),
        'coordinates': geometry.get('coordinates', [])
    })

with open('parsed_districts.json', 'w') as f:
    json.dump(parsed_data, f, indent=4)

print("Parsed data saved to 'parsed_districts.json'")

## Step 3: Generate and Store Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import json
import numpy as np

# Load parsed data
with open('parsed_districts.json', 'r') as f:
    data = json.load(f)

descriptions = [
    f"District: {item['district_name']}, Water: {item['water']}, Medkits: {item['medkits']}"
    for item in data
]

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = np.array([model.encode(desc) for desc in descriptions])

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
faiss.write_index(index, 'district_index.faiss')

with open('district_embeddings_with_metadata.json', 'w') as f:
    json.dump({'metadata': data, 'embeddings': embeddings.tolist()}, f, indent=4)

print("FAISS index and metadata saved.")

## Step 4: Query System with FAISS

In [ ]:
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer

# Load index and metadata
index = faiss.read_index('district_index.faiss')
with open('parsed_districts.json', 'r') as f:
    metadata = json.load(f)

model = SentenceTransformer('all-MiniLM-L6-v2')

def query_faiss(query, top_k=5):
    query_embedding = model.encode(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)
    return [metadata[i] for i in indices[0]], distances[0]

query = "District with high water and medkits"
results, distances = query_faiss(query)

for i, result in enumerate(results):
    print(f"Result {i+1}: {result['district_name']} (Distance: {distances[i]:.4f})")

## Step 5: RAG with Groq API

In [ ]:
import requests

GROQ_API_KEY = "your_groq_api_key_here"
GROQ_API_URL = "https://api.groq.com/v1/chat"

context = "\n".join([
    f"District: {res['district_name']}, Water: {res['water']}, Medkits: {res['medkits']}"
    for res in results
])

payload = {
    "messages": [
        {"role": "system", "content": "Provide district insights based on user query."},
        {"role": "user", "content": f"Query: {query}\n\nContext:\n{context}"}
    ]
}

headers = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}

response = requests.post(GROQ_API_URL, json=payload, headers=headers)
if response.status_code == 200:
    print("Groq Response:", response.json()["choices"][0]["message"]["content"])
else:
    print("Groq API Error:", response.text)